In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import warnings
warnings.filterwarnings('ignore')
import re


In [2]:
# connecting to the driver
driver = webdriver.Chrome(r"C:\Users\LENOVO\Downloads\chromedriver.exe")

In [3]:
# opening the website
website = "https://www.booking.com/index.en-gb.html?aid=397646&label=yho748jc-1DCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQPoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI"
driver.get(website)
time.sleep(5)

In [4]:
# XPath for the close button of the pop-up
try:
    close_button = driver.find_element(By.XPATH, '//div[@class="ffd93a9ecb dc19f70f85 eb67815534 e91f709929"]/div')
    close_button.click()
except (NoSuchElementException, ElementClickInterceptedException):
    # Handle the case when the close button is not found or cannot be clicked
    print("Failed to close the pop-up")

In [5]:
#Scroll to the bottom of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [6]:
# clicking the cities tab
driver.find_element(By.XPATH, '//div[@id="footer_links"]/div[1]/ul/li[3]/a').click()
time.sleep(10)

In [7]:
# Click to close the pop-up
driver.find_element(By.XPATH, '//button[@class="bui-popover__close"]').click()

We have to scrape data for the following cities--
1 Dubai, United Arab Emirates
2 London, United Kingdom
3 Kuala Lumpur, Malaysia
4 Manchester
5 New Delhi
6 Birmingham
7 Berlin
8 Sydney
9 Melbourne
10  Paris
11 Tokyo
12 Toronto

In [8]:
# Define the list of city names you're interested in
cities_of_interest = ["Dubai", "London", "Kuala Lumpur","Manchester", "New Delhi", "Birmingham", "Berlin",
                      "Sydney","Melbourne","Paris", "Tokyo", "Toronto"]

# Find all the city name elements
name_elements = driver.find_elements(By.XPATH, '//div[@class="block_header"]/h2/a')

# Initialize a list to store the city names
name = []

for element in name_elements:
    city_name = element.text
    # Manipulate the text to extract the city name correctly
    city_name = city_name.strip('Hotels in ').split(',')[0]
    name.append(city_name)

In [9]:
# finding the url for cities_of_interest

# Initialize a list to store city URLs
cities_url = []

# Find the URLs for cities of interest
for city_name, city_url in zip(name, driver.find_elements(By.XPATH, '//div[@class="block_header"]/h2/a')):
    if city_name in cities_of_interest:
        cities_url.append(city_url.get_attribute('href'))

In [10]:
cities_url = cities_url[:12]
cities_url

['https://www.booking.com/city/ae/dubai.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/gb/london.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/au/sydney.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/au/melbourne.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/fr/paris.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/de/berlin.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4eb728194074152dfcfa955dcb805766&inac=0&',
 'https://www.booking.com/city/gb/manchester.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=4e

In [18]:
# creating empty lists for data

Hotel_name = []
City = []
Location = []
Reviews = []
User_rating = []
Star_rating = []
Review_score = []
Free_wifi = []
Family_rooms = []
NonSmoking_rooms = []
Restaurent = []
Bar = []
Heating = []
Lift = []
Breakfast_Cuisine1 = []
Breakfast_Cuisine2 = []
Breakfast_Cuisine3 = []
Room_type = []
Single_bed = []
Double_bed = []
Prices = []

In [12]:
all_hotel_urls = []
for url in cities_url:
    driver.get(url)
    
    # Wait for the page to load successfully
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
    # Click to load hotel details
    driver.find_element(By.XPATH, '//div[@class="lp-bui-section bui-spacer--largest x2"]').click()
    
    # Loop through multiple pages (up to 40 pages)
    for page in range(40):
        try:
            url_elements = driver.find_elements(By.XPATH, '//div[@class="d6767e681c"]/h3/a')
            for element in url_elements:
                hotel = element.get_attribute('href')
                all_hotel_urls.append(hotel)
        
            try:
                next_button = driver.find_element(By.XPATH, '//div[@class="ef2dbaeb17"]/div[3]/button')
                next_button.click()
            except NoSuchElementException:
                # If the "Next" button is not found, break out of the loop
                break
        except StaleElementReferenceException:
            # Handle StaleElementReferenceException by re-locating the element
            continue

In [13]:
print(len(all_hotel_urls))

11401


In [19]:
for i in all_hotel_urls:
    driver.get(i)
    time.sleep(5)
    try:
        name = driver.find_element(By.XPATH,'//div[@id="hp_hotel_name"]/div/h2')
        Hotel_name.append(name.text)
    except NoSuchElementException:
        Hotel_name.append(None)
    
    try:
        city = driver.find_element(By.XPATH, '//p[@class="address address_clean"]/span[1]')
        city_text = city.text
        city_parts = city_text.split(',')
        if len(city_parts) >= 3:  # Ensure there are at least three parts (city, state, country)
            City.append(", ".join(city_parts[:-2]))
        else:
            City.append(None)
    except NoSuchElementException:
        City.append(None)
        
    try:
        loc = driver.find_element(By.XPATH,'//p[@class="address address_clean"]/span[1]')
        Location.append(loc.text)
    except NoSuchElementException:
        Location.append(None)
        
    try:
        rev = driver.find_element(By.XPATH,'//div[@class="aca0ade214 aaf30230d9 e1ffac4e41 e7d9f93f4d d79e71457a d5fd510f01 dc7f26e57f"]/div[2]/div[2]')
        Reviews.append(rev.text)
    except NoSuchElementException:
        Reviews.append(None)
        
    try:
        user = driver.find_element(By.XPATH,'//div[@class="aca0ade214 aaf30230d9 e1ffac4e41 e7d9f93f4d d79e71457a d5fd510f01 dc7f26e57f"]/div[2]/div')
        User_rating.append(user.text)
    except NoSuchElementException:
        User_rating.append(None)
        
    try:
        star = driver.find_element(By.XPATH, '//div[@class="a53cbfa6de bfe7b1fc60"]')
        star_text = star.text
        extracted_rating = re.search(r'\d+\.\d+', star_text).group(0) if re.search(r'\d+\.\d+', star_text) else None
        Star_rating.append(extracted_rating)
    except NoSuchElementException:
        Star_rating.append(None)
        
    try:
        score = driver.find_element(By.XPATH,'//div[@class="aca0ade214 aaf30230d9 e1ffac4e41 e7d9f93f4d d79e71457a d5fd510f01 dc7f26e57f"]/div[1]')
        Review_score.append(score.text)
    except NoSuchElementException:
        Review_score.append(None)
        
    try:
        wifi = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Free Wi-Fi")]')
        Free_wifi.append('Yes')
    except NoSuchElementException:
        Free_wifi.append('No')
        
    try:
        room = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Family rooms")]')
        Family_rooms.append('Yes')
    except NoSuchElementException:
        Family_rooms.append('No')
        
    try:
        smoke = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Non-smoking rooms")]')
        NonSmoking_rooms.append('Yes')
    except NoSuchElementException:
        NonSmoking_rooms.append('No')
        
    try:
        res = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Restaurent")]')
        Restaurent.append('Yes')
    except NoSuchElementException:
        Restaurent.append('No')
        
    try:
        bar = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Bar")]')
        Bar.append('Yes')
    except NoSuchElementException:
        Bar.append('No')
        
    try:
        heat = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Heating")]')
        Heating.append('Yes')
    except NoSuchElementException:
        Heating.append('No')
        
    try:
        lift = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Lift")]')
        Lift.append('Yes')
    except NoSuchElementException:
        Lift.append('No')
        
    try:
        room_type = driver.find_element(By.XPATH,'//div[@class="ed14448b9f b817090550 e7f103ee9e"]/div/a/span')
        Room_type.append(room_type.text)
    except NoSuchElementException:
        Room_type.append(None)
        
    try:
        bed1 = driver.find_element(By.XPATH,'//span[@class="baf7cb1417"][contains(text(), "double bed")]')
        Double_bed.append('Yes')
    except NoSuchElementException:
        Double_bed.append('No')
        
    try:
        bed2 = driver.find_element(By.XPATH,'//span[@class="baf7cb1417"][contains(text(), "single bed")]')
        Single_bed.append('Yes')
    except NoSuchElementException:
        Single_bed.append('No')
        

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00E36643]
	(No symbol) [0x00DCBE21]
	(No symbol) [0x00CCDA9D]
	(No symbol) [0x00CC9E22]
	(No symbol) [0x00CBFCFD]
	(No symbol) [0x00CC1101]
	(No symbol) [0x00CBFFDD]
	(No symbol) [0x00CBF3BC]
	(No symbol) [0x00CBF2D8]
	(No symbol) [0x00CBDC68]
	(No symbol) [0x00CBE512]
	(No symbol) [0x00CCF75B]
	(No symbol) [0x00D37727]
	(No symbol) [0x00D1FD7C]
	(No symbol) [0x00D36B09]
	(No symbol) [0x00D1FB76]
	(No symbol) [0x00CF49C1]
	(No symbol) [0x00CF5E5D]
	GetHandleVerifier [0x010AA142+2497106]
	GetHandleVerifier [0x010D85D3+2686691]
	GetHandleVerifier [0x010DBB9C+2700460]
	GetHandleVerifier [0x00EE3B10+635936]
	(No symbol) [0x00DD4A1F]
	(No symbol) [0x00DDA418]
	(No symbol) [0x00DDA505]
	(No symbol) [0x00DE508B]
	BaseThreadInitThunk [0x7658344D+18]
	RtlInitializeExceptionChain [0x77719802+99]
	RtlInitializeExceptionChain [0x777197D5+54]


In [20]:
print(len(Hotel_name), len(Location), len(City), len(Reviews), len(User_rating), len(Star_rating), len(Review_score), len(Bar),
     len(Restaurent), len(Heating), len(Lift), len(Room_type), len(Single_bed), len(Double_bed), len(Free_wifi),
      len(Family_rooms), len(NonSmoking_rooms))

675 675 675 675 675 675 675 675 675 675 675 675 675 675 675 675 675
